# Assignment 2 - Vehicle Routing Problem
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

In [194]:
# Global imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import math
import operator
from copy import deepcopy
import random
import copy
from deap import base, creator, tools, algorithms

In [195]:
# Load data
STORES = pd.read_excel('Data Excercise 2 - EMTE stores - BA 2020-1.xlsx', index_col='City Nr.')

In [196]:
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type
City Nr.,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo


## Exercise 2.1

### Calculate rounded distances between every two locations

In [197]:
def calc_dist(loc1, loc2):
    """Defines rounded distance (km) between two locations"""
    coords1 = STORES.loc[loc1, 'Lat'], STORES.loc[loc1, 'Long']
    coords2 = STORES.loc[loc2, 'Lat'], STORES.loc[loc2, 'Long']
    dist = round(haversine(coords1, coords2))
    return dist

In [198]:
# Save all distance in global variable DIST_MATRIX
DIST_MATRIX = np.zeros((len(STORES), len(STORES)))

for location_1 in STORES.index:
    for location_2 in STORES.index:
        DIST_MATRIX[location_1, location_2] = calc_dist(location_1, location_2)

### Define Location class that stores relevant data of locations

In [206]:
class Location:
    
    # Set coordinates of headquarters as class variable
    coords_hq = STORES.loc[0, 'Lat'], STORES.loc[0, 'Long']
    
    def __init__(self, nr, name, lat, long, store_type):
        self.nr = nr
        self.name = name
        self.lat = lat
        self.long = long
        self.store_type = store_type
        self.visited = False
        
    @property
    def coords(self):
        """Defines coordinates of a location"""
        return (self.lat, self.long)
    
    @property
    def distance_hq(self):
        """Defines distance from location to headquarters"""
        return DIST_MATRIX[0, self.nr]
    
    @property
    def is_jumbo(self):
        """Boolean value that shows if location is a Jumbo location"""
        return True if self.store_type=='Jumbo' else False

In [207]:
def get_params(df, index):
    """
    Function to get all necessary variables from stores dataframe to instantiate a Location instance for every store
    """
    params = df.iloc[index][['Name', 'Lat', 'Long', 'Type']].values
    name, lat, long, store_type = params
    return index, name, lat, long, store_type

### Define Route Class

In [208]:
class Route:
   
    # Set class variables
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store, hq):
        self.inner_route=[]
        self.farthest_store = farthest_store
        self.hq = hq
    
    @property
    def meeting_time(self):
        """Defines the meeting time at individiual stores in the route"""
        return 30 if self.farthest_store.is_jumbo else 20
    
    @property
    def store_types(self):
        """Defines the type of stores in this route"""
        return 'Jumbo' if self.farthest_store.is_jumbo else 'Coop or other'
    
    @property
    def full_route(self):
        """Defines full route of the route, including the start and finish at headquarters"""
        full_route = self.inner_route.copy()
        full_route.insert(0, self.hq)
        full_route.append(self.hq)
        return full_route
    
    @property
    def total_distance(self):
        """Defines total distance in km of the route"""
        return sum(Route.distances(self.full_route))
    
    @property
    def cumsum(self):
        """Defines cumulative sum of distances at each location"""
        cumsum = list(np.cumsum(Route.distances(self.full_route)))
        cumsum.insert(0, 0)
        return cumsum
    
    @property
    def total_visit_time(self):
        """Defines total time spend in all stores of route together"""
        return len(self.inner_route) * self.meeting_time
    
    @property
    def working_hours_constraint(self):
        """Defines if route meets the constraint of John's working hours"""
        return (Route.travel_time(self.full_route) + self.total_visit_time) <= Route.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
        return (Route.travel_time(self.inner_route) + self.total_visit_time) <= Route.max_opening_mins
    
    @property
    def all_hours_constraints(self):
        """Defines if route meets both constraints John's working hours and visiting hours at every store"""
        return self.working_hours_constraint and self.opening_hours_constraint
    
    @staticmethod
    def get_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        return DIST_MATRIX[loc1.nr, loc2.nr]
    
    @staticmethod
    def distances(route):
        """Defines all distances between consecutive locations in a route"""
        distances = [Route.get_dist(route[idx], route[idx+1]) for 
                          idx, _ in enumerate(route[:-1])]
        return distances
    
    @staticmethod
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (Route.speed_kmh)/60
        minutes = round(km/speed_kmm)
        return minutes
    
    @staticmethod
    def travel_time(route):
        """
        Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
        """
        minutes_between_locs = [Route.dist_to_min(dist) for dist in Route.distances(route)]
        travel_time = sum(minutes_between_locs)
        return travel_time
    
    def insert(self, pos, item):
        """Insert location at given position in route"""
        self.inner_route.insert(pos, item)
    
    def remove(self, item):
        """Remove location from route"""
        self.inner_route.remove(item)

### Define RoutePlanner class

In [209]:
class RoutePlanner:
    
    def __init__(self, locations):
        self.locations = locations[1:]
        self.hq = locations[0]
        
    @property
    def loc_distances_hq(self):
        """Defines the list of stores sorted in decreasing order based on distance from headquarter"""
        sorted_dist = sorted(self.locations, key=lambda x: x.distance_hq, reverse=True)
        return sorted_dist
    
    @property
    def to_visit(self):
        """Defines the list of stores that still need to be visited in a future route"""
        stores_to_visit = [loc for loc in self.loc_distances_hq if loc.visited==False]
        return stores_to_visit
    
    @property
    def current_farthest_store(self):
        farthest_store = [loc for loc in self.loc_distances_hq if loc.visited==False][0]
        return farthest_store
    
    @property
    def stores_left(self):
        """Defines if there are any stores left to visit"""
        return len([loc for loc in self.locations if loc.visited==False and loc!= self.hq])>0
    
    @staticmethod
    def get_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        return DIST_MATRIX[loc1.nr, loc2.nr]
    
    def potential_stores(self, farthest_store):
        """Defines potential stores for a given route with given farthest location"""
        if farthest_store.is_jumbo:
            potential_stores = [loc for loc in self.to_visit if loc.store_type=='Jumbo' and loc!=farthest_store]
        else: 
            potential_stores = [loc for loc in self.to_visit if loc.store_type!= 'Jumbo' and loc!=farthest_store]
        return potential_stores

    def dist_farthest_store(self, farthest_store):
        """Defines distances for all potential stores from farthest store in a route in sorted order ascending"""
        distances = [(RoutePlanner.get_dist(farthest_store, potential_store), potential_store) 
                     for potential_store in self.potential_stores(farthest_store)]
        distances.sort(key=operator.itemgetter(0))
        sorted_dist = [potential_store for dist, potential_store in distances]
        return sorted_dist
    
    def plan_route(self, route):
        """Plans individual routes"""
        
        # Insert current farthest store to be visited in rote
        route.insert(0, route.farthest_store)
        
        # All potential locations in ascending order
        for i in self.dist_farthest_store(route.farthest_store):
            best_dist = math.inf
            best_pos = None
            
            # Try location on every position in route
            for pos in range(len(route.inner_route)+1):
                
                # Insert location at position
                route.insert(pos, i)
                
                # Check if location at given position meets constraints and;
                # Check if this position is better than all previous viable positions for this location
                if route.all_hours_constraints and route.total_distance<best_dist:
                    best_dist = route.total_distance
                    best_pos = pos
                
                # Remove location from route
                route.remove(i)
            
            # If there is a viable position, insert location at best viable position in route
            if best_pos!=None:
                route.insert(best_pos, i)
        
        # Set all locations that are inserted to route to visited
        for loc in route.inner_route:
            loc.visited = True
            
    def solve(self):
        """Solves VRP for all locations"""
        
        # Set route number
        j = 0
        routes = {} # Empty dict to store routes
        while self.stores_left:
            j += 1 # Update route number
            farthest_store = self.current_farthest_store # Assign current farthest store
            route = Route(farthest_store, self.hq) # Instantiate route instance with current farthest store
            self.plan_route(route) # Plan this route
            routes[j] = route # Store route with route as key in dict
        return routes

### Define function to save a schedule to an Excel file

In [210]:
def to_Excel(planned_routes, exercise_nr, save=True, file_name=None):
    """Creates dataframe with needed information from solved VRP, saves to Excel file if needed"""
    
    # Dictionary to store data from all routes
    data = {'Route Nr.': [], 
            'City Nr.': [],
            'City Name': [], 
            'Total Distance in Route (km)': [],
            'Total Distance (km)': []}
    
    # Define total kilometers traveled at start of the route in previous routes
    total_km_at_start = 0
    
    # All routes in planned routes
    for nr, route in planned_routes.items():
        cumsum_loc = 0  # Define current cumulative sum location
        
        # Add data for location to dictionary
        # Because routes are instances of Route class in Exercise 1, but of RouteDLL class in Exercise 2,
        # We need to make an if-statement here
        if exercise_nr == 1:
            for location in route.full_route:
            
                data['Route Nr.'].append(nr)
                data['City Nr.'].append(location.nr)
                data['City Name'].append(location.name)
                data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
                data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
        
        # For Exercise 2, routes are instances of RouteDLL class (doubly linked lists)
        if exercise_nr == 2:
            temp = route.start # Initialise temp
            
            while (temp):
                data['Route Nr.'].append(nr)
                data['City Nr.'].append(temp.data.nr)
                data['City Name'].append(temp.data.name)
                data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
                data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
                
                temp = temp.next
            
                cumsum_loc += 1 # Update current cumulative sum location
        
        # Update total kilometers traveled at start of the route in previous routes
        total_km_at_start += route.cumsum[-1] 
    
    df = pd.DataFrame.from_dict(data) # Save data to DataFrame
    
    # If needed, save data to Excel file
    if save:
        df.to_excel(file_name, index=False)
            
    return df

In [211]:
# Instantiate location instances for each store
locations = [Location(*get_params(STORES, index)) for index, _ in STORES.iterrows()]

# Instantiate planner instance
planner = RoutePlanner(locations)
planned_routes_1 = planner.solve() # Solve VRP

# # Save results to Excel file
# df_q21 = to_Excel(planned_routes_1, 1, file_name='../excel-files/Ex2.1-1273195.xls')
print('Total distance = {}'.format(sum([route.total_distance for route in planned_routes_1.values()])))

Total distance = 2985.0


## Exercise 2.2
For exercise 2.2, we have to do a lot of swap and move operations. Therefore, for efficiency reasons, I defined another Route class - RouteDLL - that stores a route as a doubly linked list.

In [212]:
locations = [Location(*get_params(STORES, index)) for index, _ in STORES.iterrows()]

In [213]:
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

In [214]:
# Define global variable HQ
HQ = locations[0]

### Define RouteDLL Class

In [215]:
class RouteDLL:
    hq_start = Node(HQ)
    hq_end = Node(HQ)
    
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store):
        self.start = RouteDLL.hq_start
        self.end = RouteDLL.hq_end
        self.start.next = self.end
        self.end.prev = self.start
        self.farthest_store = farthest_store
        self.is_jumbo = self.farthest_store.is_jumbo
        
    @property
    def meeting_time(self):
        """Defines the meeting time at individiual stores in the route"""
        return 30 if self.farthest_store.is_jumbo else 20
    
    @property
    def length(self):
        """Defines number of stores in the route"""
 
        temp = self.start # Initialise temp 
        count = 0 # Initialise count 
  
        # Loop while end of linked list is not reached 
        while (temp): 
            count += 1
            temp = temp.next
            
        return count 
    
    @property
    def full_route(self):
        """Returns the order of stores in the full route as a list"""
        
        temp = self.start
        route = []
        
        while (temp):
            route += [temp.data.nr]
            temp = temp.next
            
        return route
    
    @property
    def total_visit_time(self):
        """Defines total time spend in all stores of route together"""
        return ((self.length)-2) * self.meeting_time
    
    @property
    def store_type_constraint(self):
        """Defines if all stores in route are of same type"""
        temp = self.start.next
        
        while temp != self.end:
            if temp.data.is_jumbo != self.is_jumbo:
                return False
            temp=temp.next
        
        return True
    
    @property
    def working_hours_constraint(self):
        """Defines if route meets the constraint of John's working hours"""
        return (self.total_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
        return (self.inner_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_opening_mins
    
    @property
    def all_hours_constraints(self):
        """Defines if route meets both constraints John's working hours and visiting hours at every store"""
        return self.working_hours_constraint and self.opening_hours_constraint
    
    @property
    def is_valid(self):
        """
        Defines if route meets all constraints, that is:
        * Constraint of John's working hours (11 hours max)
        * Constraint of visiting hours at every store (9:00-17:00, that is, 8 hours max)
        * Constraint of every store in route being either of type Jumbo or of type other (Coop or other)
        """
        return self.all_hours_constraints and self.store_type_constraint
    
    @property
    def total_distance(self):
        """Defines total distance of total route"""
        return self.total_distance_time()['total_distance']
    
    @property
    def cumsum(self):
        """Defines cumulative sum of distances at each location"""
        temp = self.start # Initialise temp
        distance = 0
        cumsum = []
        
        while temp.next:
            # Calculate distance between current and next node
            dist = RouteDLL.get_dist(temp.data, temp.next.data)
            
            # Add dist to distance
            distance += dist
            #Append cumulative distance until this node to cumsum list
            cumsum.append(distance)
            
            # Move to next node
            temp = temp.next
        
        # Insert cumsum of 0 at start at HQ
        cumsum.insert(0, 0)
        return cumsum
    
    @staticmethod
    def get_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        return DIST_MATRIX[loc1.nr, loc2.nr]
    
    @staticmethod
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (RouteDLL.speed_kmh)/60
        minutes = round(km/speed_kmm)
        return minutes
    
    def total_distance_time(self):
        """Defines total distance and travel time of total route"""
        temp = self.start # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next:
            # Calculate distance and time between current and next node
            dist = RouteDLL.get_dist(temp.data, temp.next.data)
            time = RouteDLL.dist_to_min(dist)
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
            
        return {'total_distance': distance,
                'total_travel_time': travel_time}
    
    def inner_distance_time(self):
        """Defines total distance and travel time of inner route (i.e., route without start and end at hq)"""
        temp = self.start.next # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next != self.end:
            # Calculate distance and time between current and next node
            dist = RouteDLL.get_dist(temp.data, temp.next.data)
            time = RouteDLL.dist_to_min(dist)
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
        
        return {'total_distance': distance,
                'total_travel_time': travel_time}    
        
    def insert_after(self, prev_node, new_node):
        """Insert node at given position in route"""
        
        # Check if prev_node exists
        if prev_node is None:
            print("This node doesn't exist in DLL")
            return
        
        # Make next of new node as next of prev_node
        new_node.next = prev_node.next
        
        # Make the next node of prev_node as new_node
        prev_node.next = new_node
        
        # Make the prev_node as previous of new_node
        new_node.prev = prev_node
        
        # Change previous of new_node's next node
        if new_node.next is not None:
            new_node.next.prev = new_node
            
    def remove(self, dele):
        """Remove node from route"""
        
        # Change previous pointer of dele's next node:
        dele.next.prev = dele.prev 
        
        # Change next pointer of dele's previous node:
        dele.prev.next = dele.next
        
        # Set dele's previous and next pointers to None
        dele.next=None
        dele.prev=None

#### Set Route instances of Ex 2.1 to RouteDLL instances of Ex 2.2

In [216]:
planned_routes_dll = []

for route in planned_routes_1.values():
    route_dll = RouteDLL(route.farthest_store)
    
    curr = route_dll.start # Initialise for insertion
    
    # Insert each location in inner route of route to route_dll
    for loc in route.inner_route:
        node_loc = Node(loc)
        route_dll.insert_after(curr, node_loc)
        curr = node_loc
    
    # Append a deep copy of the RouteDLL instance to a list of planned routes of type RouteDLL
    planned_routes_dll.append(deepcopy(route_dll))

### Define Solver class that can solve the local search improvement heuristic

In [217]:
class Solver:
    
    def __init__(self, planned_routes):
        self.planned_routes = planned_routes
        
    @property
    def all_routes_valid(self):
        """Checks if all planned routes are valid, i.e., meets all constraints"""
        for route in self.planned_routes:
            if not route.is_valid:
                return False
        return True
    
    @property
    def total_distance(self):
        """Defines total distance of all planned routes together"""
        dist = 0
        for route in self.planned_routes:
            dist += route.total_distance
        return dist
        
    @staticmethod
    def swap_nodes(node_1, node_2):
        """Swaps node 1 with node 2, i.e., swap data of node 1 with data of node 2"""
        # Nothing to do if x and y are same 
        if node_1 == node_2: 
            return 
    
        temp = deepcopy(node_1.data)
        
        node_1.data = node_2.data
        node_2.data = temp
        
    @staticmethod
    def move_node_after(node, node_to_move_after):
        """Move position of node to position after node_to_move_after"""
        
        # If node is is equal to node_to_move_after, do nothing
        if node == node_to_move_after:
            return
        
        # Remove node at original position
        ## Change node's previous node next pointer to node's next node
        node.prev.next = node.next
        
        ## Change node's next node previous pointer to node's previous node
        node.next.prev = node.prev
        
        # Insert node at new position
        ## Make next of new node as next of prev_node
        node.next = node_to_move_after.next
        
        ## Make the next node of prev_node as new_node
        node_to_move_after.next = node
        
        ## Make the prev_node as previous of new_node
        node.prev = node_to_move_after
        
        ## Change previous of new_node's next node
        node.next.prev = node
        
    @staticmethod
    def undo_move(route_org, route_curr, node, orig_before_node):
        """Undo the move of node"""
        
        route_curr.remove(node)
        route_org.insert_after(orig_before_node, node)
    
    def all_swaps(self, changed_routes):
        """Computes all possible swaps for all nodes in planned routes"""
        
        # Dictionary to store the swaps that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        for idx_route, route in enumerate(self.planned_routes):
            # Set node to compute all swaps with
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                # Compute only swaps with nodes in current route and routes after, in order to prevent doing double
                # the work
                for check_route in self.planned_routes[idx_route:]:
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the swap and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    # If any of the routes the two nodes are in changed, we have to recompute the improvement of the
                    # swap
                    else: 
                        if route == check_route:
                            check_loc = curr_loc.next # Only check for nodes after current node
                        else:
                            check_loc = check_route.start.next
                        
                        while check_loc != check_route.end:
                            Solver.swap_nodes(curr_loc, check_loc) # swap two nodes
                        
                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the swap leads to an improvement and results in all valid routes 
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, check_loc)] = {}
                                improvements[(curr_loc, check_loc)]['improv'] = improv
                                improvements[(curr_loc, check_loc)]['routes'] = (route, check_route)
                            
                            # Swap nodes back
                            Solver.swap_nodes(check_loc, curr_loc)
                            
                            # Move to next location to swap with
                            check_loc = check_loc.next
                            
                # Check swaps for next location
                curr_loc = curr_loc.next
        return improvements
    
    def all_moves(self, changed_routes):
        """Computes all possible swaps for all nodes in planned routes"""
        
        # Dictionary to store the moves that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        for route in self.planned_routes:
            # Set node to compute all moves for
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                orig_before_node = curr_loc.prev
                
                for check_route in self.planned_routes:
                    # Set node we will move curr_loc after
                    move_after = check_route.start
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the move and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    else:
                        while move_after != check_route.end:
                        
                            Solver.move_node_after(curr_loc, move_after) # move curr_loc after move_after

                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the move leads to an improvement and results in all valid routes,
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, move_after)] = {}
                                improvements[(curr_loc, move_after)]['improv'] = improv
                                improvements[(curr_loc, move_after)]['routes'] = (route, check_route)
                            
                            # Undo the move
                            solver.undo_move(route, check_route, curr_loc, orig_before_node)
                            
                            # Move to next location to move after
                            move_after = move_after.next
                
                # Check moves for next location
                curr_loc = curr_loc.next

        return improvements
    
    def optimize(self):
        
        operations = 0 # Initialise nr of operation to lead to optimal result
        swap_improv = {} # Dictionary to save all swap improvements
        move_improv = {} # Dictionary to save all move improvements
        
        changed_routes = self.planned_routes # Initialise routes that changed since last iteration
        
        
        while True:
            # Compute all move and swaps for this iteration
            iter_swap_improv = self.all_swaps(changed_routes)
            iter_move_improv = self.all_moves(changed_routes)
            
            # Update dictionaries with swap and move improvements with improvements of changed routes 
            # during last iteration
            swap_improv.update(iter_swap_improv)
            move_improv.update(iter_move_improv)
            
            # Sort swap improvements on improvement value decreasing
            swap_improv_list = [(values['improv'], swap, values['routes']) for swap, values in swap_improv.items()]
            swap_improv_sorted = sorted(swap_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a swap that leads to an improvement, set best_swap_improv to improvement of best swap
            if len(swap_improv_sorted) > 0:
                best_swap_improv = swap_improv_sorted[0][0]
            else:
                best_swap_improv = 0 # Else set to 0, as there is no swap improvement
            
            # Sort move improvements on improvement value decreasing
            move_improv_list = [(values['improv'], move, values['routes']) for move, values in move_improv.items()]
            move_improv_sorted = sorted(move_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a move that leads to an improvement, set best_move_improv to improvement of best move
            if len(move_improv_sorted) > 0:
                best_move_improv = move_improv_sorted[0][0]
            else:
                best_move_improv = 0
            
            # If there is no swap or move improvement anymore, we have found the optimal solution for now
            if (len(swap_improv) + len(move_improv))==0:
                break
            
            # If the best improvement is a move improvement, compute the best move
            if best_move_improv > best_swap_improv:
                self.move_node_after(*move_improv_sorted[0][1])
                operations+=1 # Increase number of operations by 1
                print('Moved {} after {}'.format(move_improv_sorted[0][1][0].data.name,
                                                 move_improv_sorted[0][1][1].data.name))
                
                # Keep track of the routes that have changed during the move operation
                changed_routes = list(move_improv_sorted[0][2])
                
                # Delete moves in changed_routes from move_improv dictionary, as they have to be computed again in 
                # next iteration
                for move in move_improv_sorted:
                    if any(True for route in changed_routes if route in move[2]):
                        del move_improv[move[1]]
                
                # Delete swaps in changed_routes from swap_improv dictionary, as they have to be computed again in 
                # next iteration
                for swap in swap_improv_sorted:
                    if any(True for route in changed_routes if route in swap[2]):
                        del swap_improv[swap[1]]
            
            # If the best improvement is a swap improvement, compute the best swap
            if best_swap_improv >= best_move_improv:
                self.swap_nodes(*swap_improv_sorted[0][1])
                operations+=1 # increase number of operations by 1
                print('Swapped {} with {}'.format(swap_improv_sorted[0][1][0].data.name,
                                                  swap_improv_sorted[0][1][1].data.name))
                
                # Keep track of the routes that have changed during the swap operation
                changed_routes = list(swap_improv_sorted[0][2])
                
                # Delete moves in changed_routes from move_improv dictionary, as they have to be computed again in 
                # next iteration
                for move in move_improv_sorted:
                    if any(True for route in changed_routes if route in move[2]):
                        del move_improv[move[1]]
                
                # Delete swaps in changed_routes from swap_improv dictionary, as they have to be computed again in 
                # next iteration
                for swap in swap_improv_sorted:
                    if any(True for route in changed_routes if route in swap[2]):
                        del swap_improv[swap[1]]
                        
        print('\nOptimized distance = {} in {} move/swap operations'.format(solver.total_distance, operations))
        
        # Make dictionary with route number and key and route instance as value
        routes = {}
        i = 1
        for route in self.planned_routes:
            routes[i] = route
            i += 1
        
        return routes

In [218]:
# Instantiate solver instance for local search improvement heuristic
solver = Solver(planned_routes_dll)
planned_routes_2 = solver.optimize() # Solve local search improvement heuristic 

# # Save results to Excel file
# df_q21 = to_Excel(planned_routes_2, 2, file_name='../excel-files/Ex2.2-1273195.xls')

Swapped EMTE HOEK FR with EMTE SEROOSKERKE FR
Moved EMTE OOLTGENSPLAAT FR after EMTE OUDENBOSCH
Swapped EMTE GROENLO with EMTE RIJSSEN VEENESLAGEN
Swapped EMTE RIJSSEN VEENESLAGEN with EMTE BATHMEN FR
Swapped EMTE UDENHOUT with EMTE KAATSHEUVEL
Moved EMTE LOON OP ZAND after EMTE HEADQUARTERS VEGHEL
Moved EMTE GROENLO after EMTE HEADQUARTERS VEGHEL
Swapped EMTE ENTER FR with EMTE RIJSSEN OPSTALLSTR
Moved EMTE Dongen after EMTE UDENHOUT
Moved EMTE TILBURG WAGNERPLEIN after EMTE UDENHOUT
Swapped EMTE OSSENDRECHT FR with EMTE PUTTE (NB)
Moved EMTE SCHAIJK after EMTE WIJCHEN
Swapped EMTE VLISSINGEN FR with EMTE KOUDEKERKE FR
Swapped EMTE ST MICHIELSGESTEL with EMTE DEN BOSCH
Moved EMTE RAVENSTEIN after EMTE HEADQUARTERS VEGHEL
Swapped EMTE LOBITH with EMTE CUIJK

Optimized distance = 2877.0 in 16 move/swap operations


## Exercise 2.3

In [329]:
STORES['is_jumbo'] = STORES.apply(lambda row: row['Type']=='Jumbo', axis=1) 
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type,is_jumbo
City Nr.,,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN,False
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop,False
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo,True
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo,True
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo,True


In [330]:
toolbox = base.Toolbox()
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.10)

In [331]:
solution_ex_2 = []

for route in planned_routes_2.values():
    temp = route.start.next
    while temp != route.end:
        solution_ex_2.append(temp.data.nr)
        temp = temp.next

solution_ex_2 = [loc-1 for loc in solution_ex_2]
        
init_pop = [deepcopy(solution_ex_2) for i in range(300)]

mut_prob = 0.95
shuffle_prob = 0.90

[toolbox.mutate(ind) if random.random() < mut_prob else ind for ind in init_pop];
[random.shuffle(ind) if random.random() < shuffle_prob else ind for ind in init_pop];

In [332]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, init_pop):
    return pcls(ind_init(ind) for ind in init_pop)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, init_pop)

In [333]:
# HELPER FUNCTIONS FOR EVALUATE FUNCTION

HQ = locations[0].nr
MAX_WORKING_MINS = 11*60
MAX_OPENING_MINS = 8*60
MAX_OPENING_MINS = 8*60
SPEED_KMH = 90

def total_distance(route, full_route=False):
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)
    
    distances = [DIST_MATRIX[route_to_check[idx], route_to_check[idx+1]] 
                 for idx, _ in enumerate(route_to_check[:-1])]
    
    return distances
    

def travel_time(route, full_route=False):
    """
    Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
    """
    
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (SPEED_KMH)/60
        minutes = round(km/speed_kmm)
        return minutes

    minutes_between_locs = [dist_to_min(dist) for dist in total_distance(route, full_route=full_route)]
    travel_time = sum(minutes_between_locs)
    return travel_time

def visiting_time(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    time = 30 if is_jumbo else 20
    return len(route)*time

def working_hours_constraint(route):
    """Defines if route meets the constraint of John's working hours"""
    return (travel_time(route, full_route=True) + visiting_time(route)) <= MAX_WORKING_MINS

def opening_hours_constraint(route):
    """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
    return (travel_time(route) + visiting_time(route)) <= MAX_OPENING_MINS

def store_type_constraint(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    
    for location in route[1:]:
        if STORES.loc[location, 'is_jumbo'] != is_jumbo:
            return False
    
    return True

def is_valid(route):
    return working_hours_constraint(route) and opening_hours_constraint(route) and store_type_constraint(route)

In [334]:
def evaluate_schedule(individual):
    schedule = []
    route = []
    for loc in individual:
        route.append(loc+1)
        if not is_valid(route):
            route.remove(loc+1)
            schedule.append(deepcopy(route))
            route = [loc+1]
    schedule.append(deepcopy(route))
    
    distances = [sum(total_distance(route, full_route=True)) for route in schedule]
    
    return sum(distances),

In [335]:
def sel_rank(individuals, k, fit_attr="fitness"):
    '''Rank based selection for a population'''
    
    # Sort individuals based on fitness value descending
    s_inds = sorted(individuals, key=operator.attrgetter('fitness.values'), reverse=True)
    
    # Define a rank for each individual based on their fitness value and define a probability distribution based on the rank
    rank = [idx+1 for idx, ind in enumerate(s_inds)]
    p_dist = [r/sum(rank) for r in rank]

    # Randomly chose k indices based, giving more weight to indices of individuals with a low fitness value through p=p_dist
    idx_choices = np.random.choice(range(len(s_inds)), size=k, p=p_dist)

    # Append individuals at indices in idx_choices to chosen
    chosen = []
    for idx in idx_choices:
        chosen.append(s_inds[idx])
        
    return chosen

In [336]:
toolbox.register("evaluate", evaluate_schedule)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", sel_rank)

In [337]:
def find_second_smallest(my_list):
    return sorted(list(set(my_list)))[1]

def find_smallest(my_list):
    return sorted(list(my_list))[0]

def mean_dist(my_list):
    return np.mean([dist for dist, nr_routes in my_list])

def std_dist(my_list):
    return np.std([dist for dist, nr_routes in my_list])

def find_largest(my_list):
    return sorted(list(my_list))[-1]

In [338]:
def main():
    pop = toolbox.population_guess()
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("2nd smallest", find_second_smallest)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=10000, 
                                   stats=stats, halloffame=hof, verbose=True)

In [339]:
main()

gen	nevals	avg    	std    	min 	max  	2nd smallest
0  	300   	13973.8	2459.62	2877	15963	(4668.0,)   
1  	220   	13589.4	2394.91	2877	15934	(4668.0,)   
2  	234   	13345.6	2134.75	5011	15489	(5739.0,)   
3  	245   	13092  	2169.41	4715	15521	(5222.0,)   
4  	215   	12682.6	2068.08	4715	15663	(5222.0,)   
5  	227   	12202.7	2048.17	4715	15643	(5222.0,)   
6  	226   	11575.7	2092.67	4715	15241	(5565.0,)   
7  	226   	10956.5	1871.85	5228	14628	(5565.0,)   
8  	227   	10554.9	1724.13	5228	14656	(5565.0,)   
9  	219   	10191.1	1583.79	5217	14092	(5228.0,)   
10 	234   	9805.21	1463.83	5228	14457	(5231.0,)   
11 	229   	9470.63	1393.71	5228	13386	(5231.0,)   
12 	245   	9375.21	1351.04	5228	13185	(5231.0,)   
13 	246   	9308.08	1292.32	5228	12350	(5231.0,)   
14 	239   	9245   	1226.26	5596	12063	(6399.0,)   
15 	230   	9190.7 	1157.24	5596	13877	(5906.0,)   
16 	234   	9133.05	1189.45	5906	12836	(6173.0,)   
17 	242   	9139.55	1149.33	5709	12609	(6199.0,)   
18 	233   	9005.28	997.145	6391

KeyboardInterrupt: 

## TRYOUT : DIFFERENT GA FOR JUMBO AND COOP

In [266]:
def calc_dist(loc1, loc2):
    """Defines rounded distance (km) between two locations"""
    dist = round(haversine(loc1.coords, loc2.coords))
    return dist

In [267]:
DIST_MATRIX = np.zeros((len(locations), len(locations)))

for location_1 in locations:
    for location_2 in locations:
        DIST_MATRIX[location_1.nr, location_2.nr] = calc_dist(location_1, location_2)

In [268]:
STORES['is_jumbo'] = STORES.apply(lambda row: row['Type']=='Jumbo', axis=1) 
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type,is_jumbo
City Nr.,,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN,False
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop,False
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo,True
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo,True
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo,True


In [269]:
solution_ex_2 = {'Jumbo': [],
                 'Coop': []}

for route in planned_routes_2.values():
    temp = route.start.next
    is_jumbo = temp.data.is_jumbo
    while temp != route.end:
        if is_jumbo:
            solution_ex_2['Jumbo'].append(temp.data.nr)
        else:
            solution_ex_2['Coop'].append(temp.data.nr)
        temp = temp.next

# solution_ex_1 = {'Jumbo': [], 
#                  'Coop': []}
        
# for route in planned_routes_1.values():
#     is_jumbo = route.inner_route[0].is_jumbo
#     for loc in route.inner_route:
#         if is_jumbo:
#             solution_ex_1['Jumbo'].append(loc.nr)
#         else:
#             solution_ex_1['Coop'].append(loc.nr)
            
            
# solution_ex_2 = [loc-1 for loc in solution_ex_2]
        
# init_pop = [deepcopy(solution_ex_2) for i in range(300)]
# shuffle_chance = 0.95

# [random.shuffle(ind) if random.random() < shuffle_chance else ind for ind in init_pop];

In [270]:
# jumbos_ex_1 = solution_ex_1['Jumbo']
# coops_ex_1 = solution_ex_1['Coop']

jumbos_ex_2 = solution_ex_2['Jumbo']
coops_ex_2 = solution_ex_2['Coop']

In [271]:
# jumbos_ex_1 = list(zip(jumbos_ex_1, [nr for nr in range(len(jumbos_ex_1))]))
# coops_ex_1 = list(zip(coops_ex_1, [nr for nr in range(len(coops_ex_1))]))

jumbos_ex_2 = list(zip(jumbos_ex_2, [nr for nr in range(len(jumbos_ex_2))]))
coops_ex_2 = list(zip(coops_ex_2, [nr for nr in range(len(coops_ex_2))]))

In [272]:
# JUMBO_GA_NR_1 = {ga_nr: nr for nr, ga_nr in jumbos_ex_1}
# COOP_GA_NR_1 = {ga_nr: nr for nr, ga_nr in coops_ex_1}

JUMBO_GA_NR = {ga_nr: nr for nr, ga_nr in jumbos_ex_2}
COOP_GA_NR = {ga_nr: nr for nr, ga_nr in coops_ex_2}

In [273]:
# jumbos_ex_1 = [ga_nr for nr, ga_nr in jumbos_ex_1]
# coops_ex_1 = [ga_nr for nr, ga_nr in coops_ex_1]

jumbos_ex_2 = [ga_nr for nr, ga_nr in jumbos_ex_2]
coops_ex_2 = [ga_nr for nr, ga_nr in coops_ex_2]

In [274]:
# HELPER FUNCTIONS FOR EVALUATE FUNCTION

HQ = locations[0].nr
MAX_WORKING_MINS = 11*60
MAX_OPENING_MINS = 8*60
MAX_OPENING_MINS = 8*60
SPEED_KMH = 90

def total_distance(route, full_route=False):
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)
    
    distances = [DIST_MATRIX[route_to_check[idx], route_to_check[idx+1]] 
                 for idx, _ in enumerate(route_to_check[:-1])]
    
    return distances
    

def travel_time(route, full_route=False):
    """
    Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
    """
    
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (SPEED_KMH)/60
        minutes = round(km/speed_kmm)
        return minutes

    minutes_between_locs = [dist_to_min(dist) for dist in total_distance(route, full_route=full_route)]
    travel_time = sum(minutes_between_locs)
    return travel_time

def visiting_time(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    time = 30 if is_jumbo else 20
    return len(route)*time

def working_hours_constraint(route):
    """Defines if route meets the constraint of John's working hours"""
    return (travel_time(route, full_route=True) + visiting_time(route)) <= MAX_WORKING_MINS

def opening_hours_constraint(route):
    """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
    return (travel_time(route) + visiting_time(route)) <= MAX_OPENING_MINS

def is_valid(route):
    return working_hours_constraint(route) and opening_hours_constraint(route)

In [275]:
def evaluate_schedule_jumbo(individual):
    schedule = []
    route = []
    for loc in individual:
        loc = JUMBO_GA_NR[loc]
        route.append(loc)
        if not is_valid(route):
            route.remove(loc)
            schedule.append(deepcopy(route))
            route = [loc]
    schedule.append(deepcopy(route))
    
    distances = [sum(total_distance(route, full_route=True)) for route in schedule]

    return sum(distances),

In [294]:
def evaluate_schedule_coop(individual):
    schedule = []
    route = []
    for loc in individual:
        loc = COOP_GA_NR[loc]
        route.append(loc)
        if not is_valid(route):
            route.remove(loc)
            schedule.append(deepcopy(route))
            route = [loc]
    schedule.append(deepcopy(route))
    
    distances = [sum(total_distance(route, full_route=True)) for route in schedule]

    return sum(distances),

In [292]:
evaluate_schedule_jumbo(jumbos_ex_2)

(1552.0,)

In [295]:
evaluate_schedule_coop(coops_ex_2)

(1350.0,)

### GA Jumbo

In [245]:
toolbox = base.Toolbox()
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.10)

In [246]:
init_pop_jumbo = [deepcopy(jumbos_ex_2) for i in range(300)]

mut_prob = 0.95
shuf_prob = 0.50
[toolbox.mutate(ind) if random.random() < mut_prob else ind for ind in init_pop_jumbo];
[random.shuffle(ind) if random.random() < shuf_prob else ind for ind in init_pop_jumbo];

In [247]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, init_pop):
    return pcls(ind_init(ind) for ind in init_pop)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, init_pop_jumbo)

In [71]:
def sel_rank(individuals, k, fit_attr="fitness"):
    '''Rank based selection for a population'''
    
    # Sort individuals based on fitness value descending
    s_inds = sorted(individuals, key=operator.attrgetter('fitness.values'), reverse=True)
    
    # Define a rank for each individual based on their fitness value and define a probability distribution based on the rank
    rank = [idx+1 for idx, ind in enumerate(s_inds)]
    p_dist = [r/sum(rank) for r in rank]

    # Randomly chose k indices based, giving more weight to indices of individuals with a low fitness value through p=p_dist
    idx_choices = np.random.choice(range(len(s_inds)), size=k, p=p_dist)

    # Append individuals at indices in idx_choices to chosen
    chosen = []
    for idx in idx_choices:
        chosen.append(s_inds[idx])
        
    return chosen

In [59]:
toolbox.register("evaluate", evaluate_schedule_jumbo)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", sel_rank)

In [60]:
def find_second_smallest(my_list):
    return sorted(list(set(my_list)))[1]

In [61]:
def main():
    pop = toolbox.population_guess()
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("2nd smallest", find_second_smallest)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, 
                                   stats=stats, halloffame=hof, verbose=True)

In [62]:
main()

gen	nevals	avg    	std    	min 	max 	2nd smallest
0  	300   	5596.41	2002.92	1552	8358	(2082.0,)   
1  	242   	4976.97	1595.13	1552	8067	(1925.0,)   
2  	239   	4704.89	1136.01	1552	8206	(1930.0,)   
3  	255   	4728.06	943.478	2082	8069	(2145.0,)   
4  	253   	4692.8 	815.347	2476	6717	(2695.0,)   
5  	253   	4771.29	750.256	2476	6789	(2695.0,)   
6  	259   	4837.57	747.021	2476	6619	(2696.0,)   
7  	239   	4910.77	671.293	2740	6486	(2906.0,)   
8  	246   	4988.43	709.747	2740	6999	(2906.0,)   
9  	246   	5102.83	710.399	2919	7111	(2980.0,)   
10 	241   	5138.67	670.974	3068	7189	(3469.0,)   
11 	243   	5235.4 	633.029	3646	6942	(3829.0,)   
12 	244   	5338.88	627.15 	3646	7461	(3949.0,)   
13 	243   	5361.23	639.771	3949	7050	(4002.0,)   
14 	257   	5510.49	624.979	4014	7441	(4019.0,)   
15 	240   	5543.19	602.197	4051	7405	(4130.0,)   
16 	249   	5602.28	587.975	4306	6971	(4320.0,)   
17 	230   	5612.6 	607.105	4320	7360	(4345.0,)   
18 	241   	5627.77	587.752	4318	7322	(4345.0,)   


KeyboardInterrupt: 

### GA Coop

In [248]:
toolbox = base.Toolbox()
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.10)

In [249]:
init_pop_coop = [deepcopy(coops_ex_2) for i in range(300)]

mut_prob = 0.50
shuf_prob = 0.95
[toolbox.mutate(ind) if random.random() < mut_prob else ind for ind in init_pop_coop];
[random.shuffle(ind) if random.random() < shuf_prob else ind for ind in init_pop_coop];

In [250]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, init_pop):
    return pcls(ind_init(ind) for ind in init_pop)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, init_pop_coop)

In [251]:
def sel_rank(individuals, k, fit_attr="fitness"):
    '''Rank based selection for a population'''
    
    # Sort individuals based on fitness value descending
    s_inds = sorted(individuals, key=operator.attrgetter('fitness.values'), reverse=True)
    
    # Define a rank for each individual based on their fitness value and define a probability distribution based on the rank
    rank = [idx+1 for idx, ind in enumerate(s_inds)]
    p_dist = [r/sum(rank) for r in rank]

    # Randomly chose k indices based, giving more weight to indices of individuals with a low fitness value through p=p_dist
    idx_choices = np.random.choice(range(len(s_inds)), size=k, p=p_dist)

    # Append individuals at indices in idx_choices to chosen
    chosen = []
    for idx in idx_choices:
        chosen.append(s_inds[idx])
        
    return chosen

In [252]:
toolbox.register("evaluate", evaluate_schedule_coop)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", sel_rank)

In [253]:
def find_second_smallest(my_list):
    return sorted(list(set(my_list)))[1]

In [254]:
def main():
    pop = toolbox.population_guess()
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("2nd smallest", find_second_smallest)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.3, mutpb=0.3, ngen=10000, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

In [255]:
pop, log, hof = main()

gen	nevals	avg   	std    	min 	max 	2nd smallest
0  	300   	5014.5	687.968	1350	5920	(1612.0,)   
1  	143   	4848.2	631.897	1350	5743	(1907.0,)   
2  	157   	4675.05	718.119	1350	5633	(2335.0,)   
3  	169   	4587.64	664.047	1350	5748	(1474.0,)   
4  	147   	4423.61	708.894	1350	5437	(1474.0,)   
5  	161   	4260.66	718.114	1695	5499	(2111.0,)   
6  	129   	3944.02	729.755	1695	5756	(2111.0,)   
7  	159   	3594.11	725.769	1608	5453	(1695.0,)   
8  	142   	3313.73	676.77 	1608	5005	(1695.0,)   
9  	147   	3085.02	652.624	1695	4737	(1817.0,)   
10 	156   	2920.8 	563.969	1695	4468	(1765.0,)   
11 	142   	2736.96	519.871	1627	4103	(1695.0,)   
12 	162   	2657.74	523.111	1695	4401	(1754.0,)   
13 	147   	2502.22	510.157	1680	4217	(1695.0,)   
14 	162   	2388.75	485.558	1474	4105	(1483.0,)   
15 	132   	2295.06	481.437	1474	3960	(1483.0,)   
16 	165   	2260.76	529.194	1474	4528	(1483.0,)   
17 	152   	2151.08	487.783	1474	3719	(1493.0,)   
18 	161   	2112.59	483.044	1459	4088	(1474.0,)   
19 

KeyboardInterrupt: 

## TRYOUT: PER SUB-ROUTE

In [161]:
sub_route_1 = []

sub_route = list(planned_routes_2.values())[0]
temp = sub_route.start.next
while temp != sub_route.end:
    sub_route_1.append(temp.data.nr)
    temp = temp.next
    
sub_route_1

[7, 40, 41, 34, 96, 75, 127, 48, 65, 79, 27, 126, 78]

In [162]:
nr_deap = {idx: nr for idx, nr in enumerate(sub_route_1)} # Dictionary to go from nr in deap to actual location number
sub_route_1_deap = [idx for idx, nr in enumerate(sub_route_1)] # Route representation for deap

In [163]:
init_pop = [deepcopy(sub_route_1_deap) for i in range(300)]

[random.shuffle(ind) for ind in init_pop];

In [164]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, init_pop):
    return pcls(ind_init(ind) for ind in init_pop)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, init_pop)

In [165]:
# HELPER FUNCTIONS FOR EVALUATE FUNCTION

HQ = locations[0].nr
MAX_WORKING_MINS = 11*60
MAX_OPENING_MINS = 8*60
MAX_OPENING_MINS = 8*60
SPEED_KMH = 90

def ind_to_subroute(ind):
    return [nr_deap[nr] for nr in ind]

def total_distance(route, full_route=False):
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)
    
    distances = [DIST_MATRIX[route_to_check[idx], route_to_check[idx+1]] 
                 for idx, _ in enumerate(route_to_check[:-1])]
    
    return distances
    

def travel_time(route, full_route=False):
    """
    Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
    """
    
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (SPEED_KMH)/60
        minutes = round(km/speed_kmm)
        return minutes

    minutes_between_locs = [dist_to_min(dist) for dist in total_distance(route, full_route=full_route)]
    travel_time = sum(minutes_between_locs)
    return travel_time

def visiting_time(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    time = 30 if is_jumbo else 20
    return len(route)*time

def working_hours_constraint(route):
    """Defines if route meets the constraint of John's working hours"""
    return (travel_time(route, full_route=True) + visiting_time(route)) <= MAX_WORKING_MINS

def opening_hours_constraint(route):
    """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
    return (travel_time(route) + visiting_time(route)) <= MAX_OPENING_MINS

def is_valid(route):
    return working_hours_constraint(route) and opening_hours_constraint(route)

In [166]:
def evaluate_subroute(individual):
    
    sub_route = ind_to_subroute(individual)
    
    schedule = []
    route = []
    
    for loc in sub_route:
        route.append(loc)
        if not is_valid(route):
            route.remove(loc)
            schedule.append(deepcopy(route))
            route = [loc]
    schedule.append(deepcopy(route))
    
    distances = [sum(total_distance(route, full_route=True)) for route in schedule]
    
    return sum(distances),

In [167]:
toolbox.register("evaluate", evaluate_subroute)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [168]:
def main():
    pop = toolbox.population_guess()
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.2, mutpb=0.7, ngen=100, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

In [169]:
pop, log, hof = main()

gen	nevals	avg    	std    	min	max 
0  	300   	923.237	60.6791	540	1044
1  	227   	885.737	61.2977	540	1014
2  	222   	855.02 	66.454 	540	1031
3  	231   	839.597	79.4694	540	1042
4  	221   	801.933	73.2736	530	1018
5  	211   	780.307	80.7737	530	976 
6  	230   	762.853	98.9397	519	1026
7  	235   	735.19 	105.672	519	960 
8  	217   	716.243	121.67 	500	978 
9  	222   	688.07 	131.984	494	1013
10 	224   	666.213	131.477	494	974 
11 	231   	649.87 	136.002	494	992 
12 	236   	636.64 	136.077	494	1018
13 	221   	627.027	138.183	468	962 
14 	224   	595.843	126.675	468	960 
15 	232   	602.27 	130.418	462	922 
16 	226   	597.2  	133.489	462	938 
17 	237   	589.6  	133.455	461	987 
18 	230   	583.583	136.975	461	997 
19 	224   	567.003	128.037	461	932 
20 	222   	565.61 	130.732	461	945 
21 	228   	570.393	138.606	461	947 
22 	229   	558.65 	138.453	458	966 
23 	230   	547.417	128.043	458	913 
24 	233   	550.497	139.245	456	887 
25 	219   	531.473	126.838	455	993 
26 	230   	539.917	132.976	4

In [170]:
best_route = [loc for route in hof.items for loc in route]
best_route = ind_to_subroute(best_route)
best_route

[78, 126, 27, 127, 75, 96, 34, 48, 79, 65, 40, 41, 7]